In [1]:
# Imports
import os
import sys
from pathlib import Path

# Reduce parallelism for stability in notebooks
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
try:
    import torch
    torch.set_num_threads(1)
except Exception:
    pass

# Resolve project root and ensure it's on sys.path
ROOT = Path.cwd().resolve()
for _ in range(5):
    if (ROOT / "pyproject.toml").exists() or (ROOT / "raw_data").exists():
        break
    ROOT = ROOT.parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))


from utils import discretize_preprocess


In [2]:
# Preprocess data
from pathlib import Path

dataset_path = ROOT / "raw_data" / "car.csv"
output_path = ROOT / "discretized_data" / "car.csv"
output_path.parent.mkdir(parents=True, exist_ok=True)

discretize_preprocess(str(dataset_path), str(output_path))



Preprocessing: /Users/vikumdabare/Documents/Work/Katabatic/katabatic/raw_data/car.csv
Saved preprocessed discrete dataset to: /Users/vikumdabare/Documents/Work/Katabatic/katabatic/discretized_data/car.csv


In [3]:
from katabatic.pipeline.train_test_split.pipeline import TrainTestSplitPipeline
from katabatic.models.great.models import GReaT

# Run pipeline
input_csv = str(output_path)
output_dir = str(ROOT / "sample_data" / "car")
real_test_dir = output_dir

synthetic_dir = str(ROOT / "synthetic" / "car" / "great")

# Use a tiny model and minimal training for stability
pipeline = TrainTestSplitPipeline(
    model=lambda: GReaT(
        llm='sshleifer/tiny-gpt2',
        epochs=1,
        batch_size=1,
        report_to=[],
    )
)
result = pipeline.run(
    input_csv=input_csv,
    output_dir=output_dir,
    synthetic_dir=synthetic_dir,
    real_test_dir=real_test_dir,
)
print(result)



/Users/vikumdabare/Documents/Work/Katabatic/katabatic/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loaded data with shape: (1728, 7)
Saved train/test full data
Train size: (1382, 7), Test size: (346, 7)
Train label distribution:
 6
2    0.700434
0    0.222142
1    0.039797
3    0.037627
Name: proportion, dtype: float64
Test label distribution:
 6
2    0.699422
0    0.222543
1    0.040462
3    0.037572
Name: proportion, dtype: float64
Saved X/y split
Training shape: (1382, 6) (1382,)
Test shape: (346, 6) (346,)


/Users/vikumdabare/Documents/Work/Katabatic/katabatic/katabatic/models/great/models.py:264: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `GReaTTrainer.__init__`. Use `processing_class` instead.
  great_trainer = GReaTTrainer(
/Users/vikumdabare/Documents/Work/Katabatic/katabatic/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


RuntimeError: [enforce fail at inline_container.cc:664] . unexpected pos 895508224 vs 895508116